In [1]:
%run /home/bb/src/python/valkyrie/lib/valkyrie/apps/common/run_notebook_import.py

/media/veracrypt1/python/valkyrie/lib/valkyrie/apps/common/run_notebook_import.py:16: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


/media/veracrypt1/python/valkyrie/lib/valkyrie/ml/data.py:291: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


In [ ]:
json_input = '''
{
    "sdate": "2023-01-01",
    "edate": "2023-01-31",
    "instrs": "BDM.BTC.USDT.FP",
    "freq": "1s",
    "ret_n_s": [60, 120, 300, 600],
    "root_dir": "/home/bb/data/BDM",    
    "fitting_param": {
        "train_ratio": 0.8,
        "model" : "TORCH:ANET",        
        "L": 1e-4,
        "B": 2048,
        "H": 8,
        "W": 64,
        "xcol": ["bpx_last","apx_last", "bq_last","aq_last","buy_qty_sum", "sell_qty_sum"],
        "ycol": "mid_last_ret_300_n"
        "epochs" : 
    }
}
'''

def train_anet(name, df_in, params, run_dir = '/home/bb/src/python/valkyrie/fitting_results/'):
    out_dir = f'{run_dir}/{name}'    
    os.system('mkdir -p {out_dir}')
    
    # Extract parameters from the input dictionary        
    lr = params["L"]
    n_channels = len(params["xcol"])
    H = params["H"]
    W = params["W"]
    D = params["D"]
    ycol = params['ycol']
    epochs = params["epochs"]

    # Create and return the neural network object
    alex_reg_net = AlexRegNet(lr=lr, num_channels=n_channels, height=H, down_mul=D)
        
    df2t2 = Df2T2(df_in, H, W, xcols = channels, ycol = ycol, yscaler = 1.0)
    train_set, val_set = TrainValDataSet.create_train_valiation_pair(df2t2, 0.8)
    dm = DataModule.from_dataset(train_set, val_set, 2048)
    
    trainer = ml_modules.Trainer(max_epochs=epochs, log_fn = f{'{out_dir}/train_{name}.log'}, plot = False)
    with Timer('training'):
        trainer.fit(alex_reg_net1, dm)
        
    return

    
    

def create_data_module_from_json(json_input):
    # Parse the JSON input
    data = json.loads(json_input)

    # Extract parameters from JSON
    sdate = data['sdate']
    edate = data['edate']
    freq = data['freq']
    root_dir = data['root_dir']
    instrs = data['instr']
    ret_n_s = data['ret_n_s']
    channels = data['channels']

    # Extract data parameters
    fitting_param = data['fitting_param']
    train_val_split = fitting_param['train_ratio']
    B = fitting_param['B']
    H = fitting_param['H']
    W = fitting_param['W']
    ycol = fitting_param['Y']

    # Data processing as per the original code
    data_mgr = DataMgr(sdate, edate, freq, root_dir, instrs=instrs, ret_n_s=ret_n_s)
    df_res = data_mgr.get(instrs[0])
    df2t2 = Df2T2(df_res, H, W, xcols=channels, ycol=ycol, yscaler=1)
    train_set, val_set = TrainValDataSet.create_train_valiation_pair(df2t2, train_val_split)
    dm = DataModule.from_dataset(train_set, val_set, batch_size)

    return dm


In [ ]:
def fit_cnn(instr, sdate, edate, freq, )